###Affine Gap Support

In [6]:
# Performs a backtrace of strings s1 and s2 using matrix s
def Backtrace(s,s1,s2):
  i = len(s1)
  j = len(s2)
  v = ""
  w = ""
  r_shift = 2
  while not (i == 0 and j == 0):

    r_shift, i_shift, j_shift, val = s[r_shift][i][j]

    if i_shift == 0 and j_shift == 0:
      continue

    if i_shift == -1 and j_shift == 0: # top
      v = s1[i-1] + v
      w = "-" + w
      i = i-1
    elif i_shift == -1 and j_shift == -1: # diag
      v = s1[i-1] + v
      w = s2[j-1] + w
      i = i-1
      j = j-1
    else: #left
      w = s2[j-1] + w
      v = "-" + v
      j = j-1

  return v,w

In [7]:
# Creates the scoreing function delta used in the recurrence
def get_delta():
  l = ["A","T","C","G","-"]
  to_return =  {}

  indel_score = -1 # -2 # gap
  match_score = 1
  mismatch_score = -1

  for a in l:
    to_return[a] = {}
    for b in l:
      if a == "-" and b == "-":
        to_return[a][b] = float("-inf")
      elif a == b:
        to_return[a][b] = match_score
      elif a == "-" or b == "-":
        to_return[a][b] = indel_score
      else:
        to_return[a][b] = mismatch_score

  return to_return

# Finds the max value between the zero_value, deletion_value,
# insertion_value, and the match_value
# Returns the maximum value and the associated backpointers
def find_max(zero_value, deletion_value, insertion_value, match_value):
    max_value = max(zero_value, deletion_value, insertion_value, match_value)

    if deletion_value == max_value:
        return 1, 0, 0, max_value
    elif insertion_value == max_value:
        return 0, 0, 0, max_value
    elif match_value == max_value:
        return 2, -1, -1, max_value
    else: # Zero value
        return 2, 0, 0, max_value

# Performs the Needleman-Wunsch recurrence for global alignment
# returns the resulting alignments
def NeedlemanWunsch(v, w, sigma, rho):

    s = [ [[0 for j in range(len(w) + 1)] for i in range(len(v) + 1)] for k in range(3)]

    delta = get_delta()

    for i in range(len(v) + 1):
        for j in range(len(w) + 1):
            for k in range(3):
                zero_value = -float('inf')
                deletion_value = -float('inf')
                insertion_value = -float('inf')
                match_value = -float('inf')

                if k == 0: # Insertion, Gap in first (right arrow)

                    if j > 1:
                        insertion_value = s[0][i][j - 1][3] - sigma
                    if j > 0:
                        match_value = s[2][i][j - 1][3] - (sigma + rho)

                    if insertion_value > match_value:
                      s[0][i][j] = (0, 0, -1, insertion_value)
                    else:
                      s[0][i][j] = (2, 0, -1, match_value)

                elif k == 1: # Deletion, Gap in second (down arrow)

                    if i > 1:
                        deletion_value = s[1][i - 1][j][3] - sigma
                    if i > 0:
                        match_value = s[2][i - 1][j][3] - (sigma + rho)

                    if deletion_value > match_value:
                      s[1][i][j] = (1, -1, 0, deletion_value)
                    else:
                      s[1][i][j] = (2, -1, 0, match_value)

                else: # Match (diagonal arrow)

                    if i == 0 and j == 0:
                        zero_value = 0
                    if i > 0:
                        deletion_value = s[1][i][j][3]
                    if j > 0:
                        insertion_value = s[0][i][j][3]
                    if i > 0 and j > 0:
                        match_value = s[2][i - 1][j - 1][3] + delta[v[i-1]][w[j-1]]

                    r_shift, i_shift, j_shift, max_value = find_max(zero_value, deletion_value, insertion_value, match_value)

                    s[2][i][j] = (r_shift, i_shift, j_shift, max_value)

    return Backtrace(s,v,w)

### Tests

In [8]:
# Tests for Needleman-Wunsch algorithm using rho = 0

# Test 1
v = "GCATCGATGGGATCGATGGA"
w = "GTCAGTCAGTTTTAGCTAG"
s1, s2 = NeedlemanWunsch(v, w, sigma=1, rho=0)

s3 = "G-CA-TCGATGGGATCGATGGA"
s4 = "GTCAGTC-A-GTTTTAGCTAG-"
if (s1 == s3 and s2 == s4) or (s1 == s4 and s2 == s3):
    print("Test 1 Pass")
else:
    print("Test 1 Fail")


# Test 2
v = "AAACTGGATCTTCGTA"
w = "AGTCCCTTGACAGCTAGCTT"
s1, s2 = NeedlemanWunsch(v, w, sigma=1, rho=0)

s3 = "AGTCCCTTGACAGCTAGCTT-"
s4 = "AAAC--TGGAT--CTT-CGTA"

if (s1 == s3 and s2 == s4) or (s1 == s4 and s2 == s3):
    print("Test 2 Pass")
else:
    print("Test 2 Fail")

# Test 3
v = "TCATTTCAGTCTTTACGAGC"
w = "TTTTCAGTCTTTCAGCGATT"
s1, s2 = NeedlemanWunsch(v, w, sigma=1, rho=0)

s3 = "T--TTTCAGTCTTTCAGCGATT"
s4 = "TCATTTCAGTCTTT-A-CGAGC"

if (s1 == s3 and s2 == s4) or (s1 == s4 and s2 == s3):
    print("Test 3 Pass")
else:
    print("Test 3 Fail")

Test 1 Pass
Test 2 Pass
Test 3 Pass


In [9]:
import random

# Counts the number of continuous gaps
# returns the resulting counts
def count_gaps(s):
  gap = False

  count = 0
  for c in s:
    if not gap and c == "-":
      count += 1
      gap = True

    if gap and c != "-":
      gap = False

  return count

# Creates n recurrences of length l
# prints the number of continuous gaps
# for rho = 0 and rho = 10
def test_affine(n,l):
  alph = ["A","T","C","G"]

  count_wun = 0
  count_aff = 0

  for n in range(n):
    cur_v = ""
    cur_w = ""
    for l in range(l):
      idx = random.randint(0,3)
      cur_v = cur_v + alph[idx]

      idx = random.randint(0,3)
      cur_w = cur_w + alph[idx]

    v_wun, w_wun = NeedlemanWunsch(cur_v, cur_w, sigma=1, rho=0)
    v_aff, w_aff = NeedlemanWunsch(cur_v, cur_w, sigma=1, rho=10)

    count_wun += count_gaps(v_wun) + count_gaps(w_wun)
    count_aff += count_gaps(v_aff) + count_gaps(w_aff)

  print("wunsch gaps = ", count_wun," Aff gaps = ", count_aff)
  print("dif = ",count_wun - count_aff)
  print("ratio = ", count_wun / count_aff)

  return

In [10]:
n = 100
l = 1000
test_affine(n, l)

wunsch gaps =  23961  Aff gaps =  979
dif =  22982
ratio =  24.47497446373851
